# Importing necessary libraries

### Importing the libraries needed for building the model.
  - numpy to do some numerical operations
  - pandas for cleaning and other processings needed by the data
  - matplotlib for visualizing if there's a need to
  - re[regular expressions for doing some cleaning on the text data itself:
    - to remove some prefix and suffixes
    - to remove emoji like comments
    - to remove @ keyword etc
  - bs4 to scrape the the website for the slangs used by Nigerians
  - requests needed for the url of websites needed for scraping


In [36]:
from collections import Counter
from itertools import combinations
import requests
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import nltk.corpus
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Scraping process

In [2]:
url = "https://insight.ng/spice/nigerian-slangs-dictionary/"
req = requests.get(url)
soup = bs(req.text, "html.parser")
slangs = soup.find("ul", attrs = {"class": "ez-toc-list-level-3"} )

# Creating a list for slang from the first website.

In [3]:
slang_list = []
for slang in slangs.select('a'):
  slang_list.append(slang.text)
# print(slang_list)
len(slang_list)

56

In [4]:
new_slang_list = []
for word in slang_list:
  word = word.split('/')
  new_slang_list.extend(word)

In [5]:
cleaned_slang_list_1 = []
for word in new_slang_list:
  word = word.replace('\xa0', '')
  word =  word.replace('.','')
  word = word.replace(':','')
  word = re.sub("\d+", '',word)
  word = word.strip()
  cleaned_slang_list_1.append(word)

# Cleaning the texts gotten from the website for easier processing.

In [6]:
unknown = ['About Author','Latest entries','+']
for word in cleaned_slang_list_1:
  if word in unknown:
    cleaned_slang_list_1.remove(word)

### 58 Texts gotten from the first website after scraping.

In [7]:
len(cleaned_slang_list_1)

58

# Scraping process for the second website

In [8]:
slangs_2_list = []

url_2 = "https://www.skabash.com/popular-nigerian-slangs-and-their-meanings/"
req_2 = requests.get(url_2)
soup_2 = bs(req_2.text, "html.parser")
slangs_2 = soup_2.find("div", attrs={"class":"lwptoc_items lwptoc_items-visible"})

for element in slangs_2.find("div").select("div"):
  slangs_2_list.append(element.find('a').text)

In [9]:
new_slang_list_2 = []

for slang in slangs_2_list:
  slang = slang.replace('\n', '')
  slang = re.sub('\d','',slang)
  slang = slang.replace('.', '')
  new_slang_list_2.append(slang)

In [10]:
new_slang_list_2.remove("Popular Nigerian slangs that are trending")

In [11]:
new_slang_list_2 = list(set(new_slang_list_2))

cleaned_slang_list_2 = []
for word in new_slang_list_2:
  word = word.split('/')
  cleaned_slang_list_2.extend(word)

### 33 slangs gotten from the second website

In [12]:
len(cleaned_slang_list_2)

33

### Combining the two slang lists into a single list by mere list addition

In [13]:
long_list = cleaned_slang_list_1 + cleaned_slang_list_2
len(long_list)

91

In [19]:
short_list = ["really very good", "very great and nice", "pretty nice",
              "God bless", "pretty nice", "better things", "Lord is good", "Bright and beautiful", "going well",
             "soft life", "good news", "greater things"]

# SCRAPING THE **TWITTER** WEB IN SEARCH OF THE TWEETS THAT CONTAINS THE ABOVE WORDS USING __SNSCRAPE__ LIBRARY

In [14]:
!pip install snscrape

In [15]:
import snscrape.modules.twitter as snstwitter

# THE SCRAPING PROCESS

  - The list was gotten by specifying the country's location via the coordinates as shown with the variable loc.

In [16]:
loc = '9.077751, 8.6774567, 100km'
tweet_list = []
for word in long_list:
  for i, item in enumerate(snstwitter.TwitterSearchScraper('{} geocode:"{}"'.format(word, loc)).get_items()):
    if i > 150:
      break
    tweet_list.append([item.content, item.likeCount, item.user.location])
df = pd.DataFrame(tweet_list, columns = ["tweets", "likes", "location"])
df.to_csv("slang_tweets_1.csv")

C:\Users\oyeni\AppData\Local\Temp\ipykernel_18504\2310741059.py:7: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet_list.append([item.content, item.likeCount, item.user.location])


In [20]:
loc = '9.077751, 8.6774567, 100km'
tweet_list = []
for word in short_list:
  for i, item in enumerate(snstwitter.TwitterSearchScraper('{} geocode:"{}"'.format(word, loc)).get_items()):
    if i > 150:
      break
    tweet_list.append([item.content, item.likeCount, item.user.location])
df_short = pd.DataFrame(tweet_list, columns = ["tweets", "likes", "location"])
df_short.to_csv("slang_tweets_2.csv")

C:\Users\oyeni\AppData\Local\Temp\ipykernel_18504\4064895595.py:7: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweet_list.append([item.content, item.likeCount, item.user.location])


In [21]:
df_short.head(20)

,tweets,likes,location
0,@AITraderGlobal Very brilliant &amp; interesti...,0,Abuja
1,Men really love the mean girls. And I’m super ...,0,"Abuja, Nigeria"
2,@khadijaarrh U really did a very good job dear,0,"Jos, Nigeria"
3,@AITraderGlobal Very brilliant &amp; interesti...,3,"Abuja,Nigeria"
4,@NastyBlaq Thank you very much @NastyBlaq eve...,0,"Abuja, Nigeria"
5,"@brownelixir It’s so good. Bar Insecure, I can...",0,
6,@silasadedoyin @Abiodun0x 1. It’s very possibl...,0,"Abuja, Nigeria"
7,even after such a traumatic near death experie...,0,"Abuja, Nigeria"
8,@JefferyAnkamah @phil_Adesh @AkosuaAmpofowah G...,2,Federal Capital Territory
9,@con2etal 😂 the man dey give me joy shaa..I re...,1,"Abuja, Nigeria"


In [22]:
df_short['label'] = 'not vulgar'
df['label'] = 'vulgar'

In [23]:
df_total = pd.concat([df, df_short], axis=0, ignore_index=True)

### Making a copy of the dataset

In [24]:
df_1 = df_total.copy()

# **CLEANING THE TEXT DATA**
### The Text data needs to be cleaned before being fed into the model for training and testing.
    - Normalizing 
    - Remove Unicode Characters
    - Remove Stopwords
    - Perform Stemming
    - Lemmatization

**Normalizing the texts..**
  - changing all to lowercases also known as _case normalization_
    - creating a function to used for changing the case to a lowercase.

In [25]:
def lowercase(text):
  text = text.lower()
  return text

**Removing Unicode Characters**
  - Creating a function to remove unicode characters
    - This function uses regular expression library

In [26]:
def unicode_removal(text):
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
  return text

# **STOP WORDS**
#### Stopwords are the most common words in any natural language. For the purpose of analyzing text data and building NLP models, these stopwords might not add much value to the meaning of the document. Generally, the most common words used in a text are “the”, “is”, “in”, “for”, “where”, “when”, “to”, “at” etc. 

**Removing Stopwords**
  - Creating a function to remove the stop words
    - This function uses the Natural Language ToolKit library 

In [27]:
# downloading the stopwords needed
nltk.download('stopwords')

def stopwords_removal(text):
  stop = stopwords.words('english')
  text = " ".join([word for word in text.split() if word not in (stop)])
  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oyeni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Stemming

#### Stemming nvolves grouping of words by their root stem. This makes it clear or helps recognize that ‘jumping’ ‘jumps’ and ‘jumped’ are all rooted to the same verb (jump) and thus are referring to similar problems.
  - Creating a function to stem the texts.

In [28]:
def stemming(text):
  stemmer = PorterStemmer()
  text = " ".join([stemmer.stem(word) for word in text.split()])
  return text

# Lemmatization

#### Lemmatization groups words based on root definition, and helps to differentiate between present, past, and indefinite.

#### In order words, ‘jumps’ and ‘jump’ are grouped into the present ‘jump’, as different from all uses of ‘jumped’ which are grouped together as past tense, and all instances of ‘jumping’ which are grouped together as the indefinite (meaning continuing/continuous).

In [29]:
nltk.download('wordnet')
def lemmatize(text):
  lemmatizer = WordNetLemmatizer()
  text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
  return text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oyeni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Selecting the useful feature we need to train the model.

In [30]:
x = df_1['tweets']
y = df_1['label']

### Passing the text one by one through each of the functions created above using the lambda function method.

In [31]:
x = x.apply(lambda x: lowercase(x))
x = x.apply(lambda x: unicode_removal(x))
x = x.apply(lambda x: stopwords_removal(x))
x = x.apply(lambda x: stemming(x))
x = x.apply(lambda x: lemmatize(x))

In [32]:
y = y.apply(lambda y:1 if y == 'vulgar' else 0)

### Splitting the dataset into training and testing sets using the train test split...

In [33]:
x_train, x_test, y_train, y_test =  train_test_split(x, y, test_size=0.3, random_state=40)

### Using Count vectorizer to vectorize the preprocessed texts

In [34]:
vectorizer =  CountVectorizer()
x_train_vect = vectorizer.fit_transform(x_train)

In [37]:
model = GaussianNB()
model.fit(x_train_vect.toarray(),y_train)

GaussianNB()

### Testing the model

In [38]:
x_test_vect = vectorizer.transform(x_test)

In [39]:
y_pred = model.predict(x_test_vect.toarray())

## Checking the accuracy score
###    - f1 score
###    - precision
###    - recall
###    - and the _confusion matrix_

In [40]:
print("accuracy score:", accuracy_score(y_test, y_pred))

accuracy score: 0.8382196162046909


In [41]:
print('classification reports:\n',classification_report(y_test,model.predict(x_test_vect.toarray())))

classification reports:
               precision    recall  f1-score   support

           0       0.31      0.46      0.37       385
           1       0.93      0.88      0.91      3367

    accuracy                           0.84      3752
   macro avg       0.62      0.67      0.64      3752
weighted avg       0.87      0.84      0.85      3752



##### The model has 84% accuracy

### Saving the model as a pickle file

In [43]:
# saving our decision tree model using pickle

import pickle

with open('tweet.pkl','wb') as myfile:
    pickle.dump(model,myfile)

with open('tweet.pkl','rb') as myfile:
    model = pickle.load(myfile)

In [82]:
text = ["sapa is not nice o na why you dey do like mad man"]
text_vect = vectorizer.transform(text)

In [83]:
model.predict(text_vect.toarray())

array([1], dtype=int64)